In [2]:
from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from datetime import datetime, timedelta
import mysql.connector
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from dotenv import load_dotenv
import os

In [31]:
#company_df = pd.read_csv("D:\Curious\workday-scrapper\database\company_list.csv")

True

## Database Connection

In [3]:
load_dotenv()
# Establish a connection to your MySQL database
connection = mysql.connector.connect(
    host=os.getenv('DB_HOST'),
    port=3306,
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD'),
    database=os.getenv('DB_NAME')
)

# Create a cursor object to interact with the database
cursor = connection.cursor()

In [4]:
# Getting the company_info Data from the Database and storing it in a dataframe

#Fetching the company_info data from the database
get_sql_query = "SELECT * FROM company_info;"
cursor.execute(get_sql_query)
company_data = cursor.fetchall()

#Converting the fetched data into a dataframe
company_df = pd.DataFrame(company_data, columns=['company_id', 'company_name', 'company_url'])

In [41]:
company_df.head()

,company_id,company_name,company_url
0,1,Prudential Financial,https://pru.wd5.myworkdayjobs.com/en-US/Careers
1,2,Disney,https://disney.wd5.myworkdayjobs.com/disneycar...
2,3,FactSet,https://factset.wd1.myworkdayjobs.com/en-US/Fa...
3,4,American Family Insurance,https://amfam.wd1.myworkdayjobs.com/Careers
4,5,Santander,https://santander.wd3.myworkdayjobs.com/en-US/...


In [54]:
import time
# Implement your scraping logic here for the current company
job_data = []
# Assuming you have a DataFrame named 'company_df' containing company information
for index, company_row in company_df.iterrows():
    # Assuming you have retrieved the company_id, company_name, and company_url from the DataFrame
    company_id = company_row['company_id']
    company_name = company_row['company_name']
    company_url = company_row['company_url']

    # Extract the base URL from the company URL
    # if company_url.endswith('/en-US/Careers'):
    #     base_company_url = company_url[:-len('/en-US/Careers')]
    # elif company_url.endswith('/en-US'):  # Check if company_url ends with "/en-US"
    #     base_company_url = company_url[:-len('/en-US')]  # Remove "/en-US" if it exists
    # elif company_url.endswith('/'):
    #     base_company_url = company_url.rsplit('/', 1)[0]
    # else :
    #     base_company_url=company_url

    base_company_url = company_url.split('.com')[0] + '.com'

    
    # Navigate to the company's website
    driver = webdriver.Chrome()
    driver.get(company_url)


    page_count = 0
    while page_count < 3:
        # Get the page source
        page_source = driver.page_source
        
        # Parse the page source with BeautifulSoup
        soup = BeautifulSoup(page_source, 'html.parser')

        # Find all job postings on the current page
        job_postings = soup.find_all("li", class_="css-1q2dra3")

        # Extract job data from each posting
        for job in job_postings:
            try:
                job_title_element = job.find('a', class_='css-19uc56f')
                job_title = job_title_element.text.strip()
                print('jobTitle',job_title)
                job_url = job_title_element['href']
                

                location_element = job.find('dt', string='locations').find_next_sibling('dd')
                location = location_element.text.strip()
                
                
                posting_date_element = job.find('dt', string=lambda text: text in ['posted on', 'postedOn']).find_next_sibling('dd')
                posting_date = posting_date_element.text.strip()
                

                job_id_element = job.find('li', class_='css-h2nt8k')
                job_id = job_id_element.text.strip()
                

                
                if posting_date in ['Posted Today', 'Posted Yesterday']:
                    full_job_url = f"{base_company_url}{job_url}"
            
                    job_data.append({
                        'company_id': company_id,
                        'company_name': company_name,
                        'company_url': company_url,
                        'job_title': job_title,
                        'job_url':  full_job_url,
                        'location': location,
                        'posting_date': posting_date,
                        'job_id': job_id
                    })
                    
                else:
                    continue
            except AttributeError as e:
                print("Error occurred while extracting job data:", e)
                continue

        # Increment page count
        page_count += 1
        # Look for the next page button
        # next_button_xpath = f'//li[contains(@class, "css-1j096s0")]/button[text()="{page_count + 1}"]'
        try: 
            next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//li[contains(@class, 'css-1j096s0')]/button[contains(@aria-label, 'page')]")))
            print(next_button)
            # next_button = driver.find_element(By.XPATH, next_button)

            # If there is no next page button, break the loop
            if not next_button or page_count >= 3:
                break
            # Click the next page button
            next_button.click()
            
            # Wait for the page to load
            WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, "loading-indicator")))  # Replace "loading-indicator" with actual ID if applicable
        except Exception as e:
            print(f"Error navigating to the next page: {e}")
            continue  # Break the loop if unable to find the next button or any other exception occurs

    # Quit the driver
    driver.quit()
    


<selenium.webdriver.remote.webelement.WebElement (session="c4a4854235be3bcb5fcc7058c3904d1e", element="f.9C3229FAD946AA996A8E3682AABA776F.d.40273E19BD2EAA44719DA188D9069591.e.46")>
<selenium.webdriver.remote.webelement.WebElement (session="c4a4854235be3bcb5fcc7058c3904d1e", element="f.9C3229FAD946AA996A8E3682AABA776F.d.40273E19BD2EAA44719DA188D9069591.e.46")>
<selenium.webdriver.remote.webelement.WebElement (session="c4a4854235be3bcb5fcc7058c3904d1e", element="f.9C3229FAD946AA996A8E3682AABA776F.d.40273E19BD2EAA44719DA188D9069591.e.46")>
<selenium.webdriver.remote.webelement.WebElement (session="3c33e79d88a10d86f1a6ed19f33aba69", element="f.3A5119A3B5AA3577FF9F03AD2056E738.d.8CDC05455E4B1FFFEB3B75A26127FA77.e.33")>
<selenium.webdriver.remote.webelement.WebElement (session="3c33e79d88a10d86f1a6ed19f33aba69", element="f.3A5119A3B5AA3577FF9F03AD2056E738.d.8CDC05455E4B1FFFEB3B75A26127FA77.e.33")>
<selenium.webdriver.remote.webelement.WebElement (session="3c33e79d88a10d86f1a6ed19f33aba69", e

## Taking into dataframe - all common jobs

In [50]:
df_job = pd.DataFrame(job_data)

In [46]:
df_job = df_job.drop_duplicates()

In [51]:
df_job.shape

(351, 8)

In [52]:
df_job.shape

(0, 0)

In [43]:
# Function to update posting date
def update_posting_date(row):
    if row['posting_date'] == 'Posted Today':
        return datetime.now().strftime('%Y-%m-%d')
    elif row['posting_date'] == 'Posted Yesterday':
        return (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
    else:
        return row['posting_date']

# Apply the function to update posting dates
df_job['posting_date'] = df_job.apply(update_posting_date, axis=1)

In [44]:
#Inserting all rows from df_job into job_info table
for index, row in df_job.iterrows():
    insert_query = "INSERT INTO job_info (job_id, job_title, job_url, job_location, post_date, company_id) VALUES (%s, %s, %s, %s, %s, %s)"
    job_values = (row['job_id'], row['job_title'], row['job_url'], row['location'], row['posting_date'], row['company_id'])
    cursor.execute(insert_query, job_values)

# Commit changes to the database
connection.commit()


In [20]:
df_job.columns

Index(['company_id', 'company_name', 'company_url', 'job_title', 'job_url',
       'location', 'posting_date', 'job_id'],
      dtype='object')

In [23]:
df_job.to_csv("database/job_listings.csv",index=False)

## Software jobs

In [24]:
df_job_software = df_job[df_job['job_title'].str.contains(r"\bsoftware\b", flags=re.IGNORECASE)]

In [25]:
df_job_software.to_csv("database/software_job_listings.csv",index=False)

## Data jobs

In [26]:
df_job_data = df_job[df_job['job_title'].str.contains(r"\bdata\b", flags=re.IGNORECASE)]

In [27]:
df_job_data.to_csv("database/data_job_listings.csv",index=False)

## Database conenction- Insert into MYSQL database 

In [2]:
# Establish a connection to your MySQL database
connection = mysql.connector.connect(
    host='localhost',
    port = 3306,
    user='root',
    password='Aayurajvi12!',
    database='scrapper'
)

# Create a cursor object to interact with the database
cursor = connection.cursor()

In [4]:
# Fetching all data from Database.
get_sql_query = "SELECT * FROM company_info;"
cursor.execute(get_sql_query)
company_data = cursor.fetchall()
company_df = pd.DataFrame(company_data, columns=['company_id', 'company_name', 'company_url'])


In [10]:
print(company_df.loc[0])

company_id                                                    1
company_name                               Prudential Financial
company_url     https://pru.wd5.myworkdayjobs.com/en-US/Careers
Name: 0, dtype: object


In [ ]:
for index, row in df_job.iterrows():
    # Extract data from the DataFrame
    job_id = row['job_id']
    job_title = row['job_title']
    job_url = row['job_url']
    job_location = row['location']
    posting_date = row['posting_date']
    
    # Assuming you have the company_id already defined
    company_id = 1  # You need to set the appropriate company_id 
    # Check if the job_id already exists in the job_info table
    cursor.execute("SELECT job_id FROM job_info WHERE job_id = %s", (job_id,))
    result = cursor.fetchone()
    if result:
        # print(f"Skipping job with ID {job_id} as it already exists in the database.")
        continue  # Skip this iteration of the loop

    # Insert data into the job_info table using parameterized query to avoid SQL injection
    sql = "INSERT INTO job_info (job_id, job_title, job_url, job_location, posting_date, company_id) VALUES (%s, %s, %s, %s, %s, %s)"
    values = (job_id, job_title, job_url, job_location, posting_date, company_id)
    cursor.execute(sql, values)

# Commit the transaction
connection.commit()

# # Close cursor and connection
# cursor.close()
# connection.close()


In [ ]:
for index, row in company_df.iterrows():
    # Extract company information from the DataFrame
    company_name = row['company_name']
    company_url = row['company_url']
    
    # Check if the company already exists in the company_info table
    cursor.execute("SELECT company_id FROM company_info WHERE company_name = %s", (company_name,))
    result = cursor.fetchone()
    if result:
        print(f"Skipping company {company_name} as it already exists in the database.")
        continue  # Skip this iteration of the loop
    
    # Insert company information into the company_info table
    cursor.execute("INSERT INTO company_info (company_name, company_url) VALUES (%s, %s)", (company_name, company_url))
    
# Commit the transaction
connection.commit()

# Close cursor and connection
cursor.close()
connection.close()


In [14]:
cursor.close()
connection.close()
